# 2D Acoustic Wave Equation
---

Consider the 2d acoustic wave equation
$$
\begin{aligned}
&\frac{\partial p}{\partial t} = -c^2(x, z)\rho(x, z)\left(\frac{\partial u}{\partial x} + \frac{\partial v}{\partial z}\right) + f(x, z, t) \\
&\frac{\partial u}{\partial t} = -\frac{1}{\rho(x, z)}\frac{\partial p}{\partial x} \\
&\frac{\partial v}{\partial t} = -\frac{1}{\rho(x, z)}\frac{\partial p}{\partial z},
\end{aligned}
$$
where $p$ is Pressure, $u, v$ are the $x, z-$components of Particle Velocity, $c, \rho$ are Medium Velocity and Density.

The RHS
$$
f(x, z, t) = R(t)\times 1_{\{x = x_s, z = z_s, t\}} 
$$
is the source term, represented by a Ricker wavelet
$$
R(t) = \left(1-2(\pi f_0(t-t_0))^2\right)\exp\left(-(\pi f_0(t-t_0))^2\right),
$$
where $f_0 = 50$ is the dominant frequency, $t_0 = 0.05$ is time delay.

<p style="color:red"> We can probably use the same technique as the numerical code, i.e., $f(x, z, t) = R(t)\times N(x, z)$, where $N(x, z)$ is a Gaussian centering at $(x_s, z = z_s)$ ? </p>

In this case,
$$
N(x, z) = \exp\left(-\frac{1}{\alpha^2}\left((x-x_s)^2+(z-z_s)^2\right)\right).
$$


---

## Data

- $u$ is unknown
- $v_z$: Velocity in $z-$ direction; the $v$ variable in the equation
- $p$: Pressure
- $\rho(x, z)$: Medium density; We first assume it to be known as
$$
\rho(x, z) = \left\{
\begin{aligned}
&2000, &\quad z\in [100, 150], \\
&1000, &\quad \text{otherwise}.
\end{aligned}
\right.
$$

- All known data are collected in the spatial domain $(x, z) \in [1, 250]\times [1, 250]$, with grid size 5 (which means the spatial grids are {5, 10, ..., 250}. The temporal domain is $t\in [0, 0.25)$, with time step $dt = 0.0025$. This gives 100 time steps as recorded.

- The sources are located at $(x_s, z_s) = \{25, 50, \cdots, 225\} \times \{5\}$.

## Target
- Recover the direct problem $p, u, v$
- Recover the inverse problem $c$

---

## Initialize inverse model:

- add bias (close to true value) to the last layer of inverse model

In [1]:
NAME = "0628_1_direct_model_with_source_input_siren_scale_equation_weighted_loss"

In [2]:
import tensorflow as tf
import numpy as np
import tensorflow.keras as keras
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
import time

import sys
sys.path.append("../../")
from Seismic_wave_inversion_PINN.tf_model_utils import *
from Seismic_wave_inversion_PINN.data_utils import *

plt.rcParams.update(plt.rcParamsDefault)
tf.keras.backend.set_floatx("float32")

%load_ext autoreload
%autoreload 2

In [3]:
def setup_model(layers, c, w0, lambda_1):
	with tf.device("/device:GPU:0"):
		model = keras.models.Sequential()
		model.add(keras.layers.Dense(layers[1], input_shape = (layers[0], ), activation = K.sin,
									kernel_initializer = keras.initializers.RandomUniform(-w0*np.sqrt(c/layers[0]), w0*np.sqrt(c/layers[0])),
									kernel_regularizer = keras.regularizers.l1(lambda_1)))
# 		model.add(keras.layers.BatchNormalization())
		for i in range(1, len(layers)-2):
			model.add(keras.layers.Dense(layers[i+1], input_shape = (layers[i], ), activation = K.sin,
									kernel_initializer = keras.initializers.RandomUniform(-np.sqrt(c/layers[i]), np.sqrt(c/layers[i])),
									kernel_regularizer = keras.regularizers.l1(lambda_1)))
# 			model.add(keras.layers.BatchNormalization())
		model.add(keras.layers.Dense(layers[-1]))
	return model

c = 6
w0 = 2000
lambda_1 = 0.5

direct_layers = [4, 32, 32, 32, 32, 3] # (x, z, t, xs) -> (p, u, v)
direct_model = setup_model(direct_layers, c, w0, lambda_1)

w1 = 200
inverse_layers = [2, 32, 32, 1] # (x, z) -> (c)
inverse_model = setup_model(inverse_layers, c, w1, lambda_1)

In [4]:
x0, z0, t0 = 1e3, 1e3, 1.0
p0, u0, v0 = 1.0, 1.0, 1.0
c0 = 1e3

In [5]:
@tf.function
def rho(x, z):
	return tf.where(tf.math.logical_and(tf.greater_equal(z, 100.0), tf.less_equal(z, 150.0)),
				   2000.0,
				   1000.0)

# source
f_0 = 50.0*np.pi
alpha = 2.0*5 # multiplied by dx
M0 = 1000 # scale p

@tf.function
def R(t, t_s):
	return M0*(1-2*(f_0*(t-t_s))**2)*tf.exp(-(f_0*(t-t_s))**2)

@tf.function
def N(x, z, x_s, z_s):
	return tf.exp(-1.0/(alpha**2) * ((x-x_s)**2 + (z-z_s)**2))

@tf.function
def f(x, z, t, x_s, z_s, t_s):
	return R(t, t_s)*N(x, z, x_s, z_s)

In [6]:
w_zero = 1e-2
threshold = 1e-3

def weighted_loss(true, pred):
	error = K.square(true - pred)
	error = K.mean(K.switch(K.less_equal(K.abs(true), threshold), w_zero * error , error))
	return error 

# @tf.function
# def weighted_loss(y_pred, y_true):
# # 	tf.print(tf.where(tf.less_equal(tf.abs(y_true), threshold),
# # 					w_zero*tf.abs(y_pred - y_true),
# # 					w_nonzero*tf.abs(y_pred - y_true)))
# 	return tf.reduce_mean(tf.where(tf.less_equal(tf.abs(y_true), threshold),
# 					w_zero*tf.abs(y_pred - y_true),
# 					w_nonzero*tf.abs(y_pred - y_true)))

In [7]:
loss_func = keras.losses.MeanSquaredError()
loss_func_r = keras.losses.MeanAbsoluteError() # l1 loss

def get_residue_loss(tape, p, u, v, c, x, z, t, x_s, z_s, t_s):
	dp_dt = tape.gradient(p, t)
	dp_dx = tape.gradient(p, x)
	dp_dz = tape.gradient(p, z)
	du_dt = tape.gradient(u, t)
	du_dx = tape.gradient(u, x)
	dv_dt = tape.gradient(v, t)
	dv_dz = tape.gradient(v, z)
	eq1 = loss_func_r(p0/t0*dp_dt, -c0**2*tf.square(c)*rho(x0*x, z0*z)*(u0/x0*du_dx+v0/z0*dv_dz)+f(x0*x, z0*z, t0*t, x0*x_s, z0*z_s, t0*t_s))
	eq2 = loss_func_r(u0/t0*du_dt, -1.0/rho(x0*x, z0*z)*p0/x0*dp_dx)
	eq3 = loss_func_r(v0/t0*dv_dt, -1.0/rho(x0*x, z0*z)*p0/z0*dp_dz)
	return eq1, eq2, eq3
	
def step(data_sources):
	losses = []
	with tf.GradientTape(persistent = True) as tape:
		tape.watch([x for d in data_sources for x in d])
		for data in data_sources:
			[x_c, z_c, t_c, x_s_c, z_s_c, t_s_c,
			 x_i, z_i, t_i, p_i, u_i, v_i, x_s_i, z_s_i, t_s_i,
			 x_d, z_d, t_d, p_d, v_d, x_s_d, z_s_d, t_s_d] = data
			out_c = direct_model(tf.concat([x_c, z_c, t_c, x_s_c], axis = 1))
			c_c = inverse_model(tf.concat([x_c, z_c], axis = 1))
			out_d = direct_model(tf.concat([x_d, z_d, t_d, x_s_d], axis = 1))
			c_d = inverse_model(tf.concat([x_d, z_d], axis = 1))
			out_i = direct_model(tf.concat([x_i, z_i, t_i, x_s_i], axis = 1))
			c_i = inverse_model(tf.concat([x_i, z_i], axis = 1))
			loss_c1, loss_c2, loss_c3 = get_residue_loss(tape, out_c[:, 0:1], out_c[:, 1:2], out_c[:, 2:3], c_c, x_c, z_c, t_c, x_s_c, z_s_c, t_s_c)
			loss_dr1, loss_dr2, loss_dr3 = get_residue_loss(tape, out_d[:, 0:1], out_d[:, 1:2], out_d[:, 2:3], c_d, x_d, z_d, t_d, x_s_d, z_s_d, t_s_d)
			loss_dv = weighted_loss(out_d[:, 0:1], p_d) # + loss_func(out_d[:, 2:3], v_d)
			loss_ir1, loss_ir2, loss_ir3 = get_residue_loss(tape, out_i[:, 0:1], out_i[:, 1:2], out_i[:, 2:3], c_i, x_i, z_i, t_i, x_s_i, z_s_i, t_s_i)
			loss_iv = loss_func(out_i[:, 0:1], p_i) + loss_func(out_i[:, 1:2], u_i) + loss_func(out_i[:, 2:3], v_i)
			losses.append([loss_c1, loss_c2, loss_c3, loss_dr1, loss_dr2, loss_dr3, loss_dv, loss_ir1, loss_ir2, loss_ir3, loss_iv])

		loss_c1 = sum([l[0] for l in losses])
		loss_c2 = sum([l[1] for l in losses])
		loss_c3 = sum([l[2] for l in losses])
		loss_dr1 = sum([l[3] for l in losses])
		loss_dr2 = sum([l[4] for l in losses])
		loss_dr3 = sum([l[5] for l in losses])
		loss_dv = sum([l[6] for l in losses])
		loss_ir1 = sum([l[7] for l in losses])
		loss_ir2 = sum([l[8] for l in losses])
		loss_ir3 = sum([l[9] for l in losses])
		loss_iv = sum([l[10] for l in losses])
		
		loss = w_c1*loss_c1 + w_c2*loss_c2 + w_c3*loss_c3 + w_dr1*loss_dr1 + w_dr2*loss_dr2 + w_dr3*loss_dr3 + \
				w_dv*loss_dv + w_ir1*loss_ir1 + w_ir2*loss_ir2 + w_ir3*loss_ir3 + w_iv*loss_iv
		
	grads = tape.gradient(loss, direct_model.trainable_variables + inverse_model.trainable_variables)
# 	tf.print(grads)
	optimizer.apply_gradients(zip(grads, direct_model.trainable_variables + inverse_model.trainable_variables))
	
	del tape
	return loss, loss_c1, loss_c2, loss_c3, loss_dr1, loss_dr2, loss_dr3, loss_dv, loss_ir1, loss_ir2, loss_ir3, loss_iv


def train(data_sources, epochs, batch_proportion = 0.1, print_every = 100, save_every = 10000, save_path = None):
	shuffled = lambda dataset: [tf.data.Dataset.from_tensor_slices(tuple(d)).shuffle(buffer_size = max(d[0].shape[0], 1), \
                                                        reshuffle_each_iteration = True).batch(max(int(batch_proportion*d[0].shape[0]), 1)) \
                                        for d in dataset]
# 	shuffled = lambda dataset: [tf.data.Dataset.from_tensor_slices(tuple(d)).batch(max(int(batch_proportion*d[0].shape[0]), 1)) \
#                                         for d in dataset]
	data_s = shuffled([d for d_s in data_sources for d in d_s])
	for epoch in range(1, epochs+1):
		losses = np.zeros((12,))
		for d in zip(*data_s): # d: [d_1_c, d_1_i, d_1_d, d_2_c, ...]
			data_sources = [list(d[i*3])+list(d[i*3+1])+list(d[i*3+2]) for i in range(n_sources)]
			loss = step(data_sources)
			losses += np.array(loss)
		losses *= batch_proportion
		
		if epoch % print_every == 0:
			print("{}, Epoch: {}, Loss: {:.4e}, c1: {:.4e}, c2: {:.4e}, c3: {:.4e}, dr1: {:.4e}, dr2: {:.4e}, dr3: {:.4e}, \
					  dv: {:.4e}, ir1: {:.4e}, ir2: {:.4e}, ir3: {:.4e}, iv: {:.4e}".format(get_time(), epoch, *list(losses)))

		if epoch % save_every == 0:
			direct_model.save("models/{}/{}/direct_model".format(NAME, save_path))
			inverse_model.save("models/{}/{}/inverse_model".format(NAME, save_path))
			
			
class LBFGS(LBFGS_compatible):
	def set_data(self, data_sources):
		self.data_sources = data_sources
	
	@tf.function
	def loss_function(self, tape):
		tape.watch([x for d in self.data_sources for x in d])
		losses = []
		for data in self.data_sources:
			[x_c, z_c, t_c, x_s_c, z_s_c, t_s_c,
			 x_i, z_i, t_i, p_i, u_i, v_i, x_s_i, z_s_i, t_s_i,
			 x_d, z_d, t_d, p_d, v_d, x_s_d, z_s_d, t_s_d] = data
			out_c = direct_model(tf.concat([x_c, z_c, t_c, x_s_c], axis = 1))
			c_c = inverse_model(tf.concat([x_c, z_c], axis = 1))
			out_d = direct_model(tf.concat([x_d, z_d, t_d, x_s_d], axis = 1))
			c_d = inverse_model(tf.concat([x_d, z_d], axis = 1))
			out_i = direct_model(tf.concat([x_i, z_i, t_i, x_s_i], axis = 1))
			c_i = inverse_model(tf.concat([x_i, z_i], axis = 1))
			loss_c1, loss_c2, loss_c3 = get_residue_loss(tape, out_c[:, 0:1], out_c[:, 1:2], out_c[:, 2:3], c_c, x_c, z_c, t_c, x_s_c, z_s_c, t_s_c)
			loss_dr1, loss_dr2, loss_dr3 = get_residue_loss(tape, out_d[:, 0:1], out_d[:, 1:2], out_d[:, 2:3], c_d, x_d, z_d, t_d, x_s_d, z_s_d, t_s_d)
			loss_dv = weighted_loss(out_d[:, 0:1], p_d) # + loss_func(out_d[:, 2:3], v_d)
			loss_ir1, loss_ir2, loss_ir3 = get_residue_loss(tape, out_i[:, 0:1], out_i[:, 1:2], out_i[:, 2:3], c_i, x_i, z_i, t_i, x_s_i, z_s_i, t_s_i)
			loss_iv = loss_func(out_i[:, 0:1], p_i) + loss_func(out_i[:, 1:2], u_i) + loss_func(out_i[:, 2:3], v_i)
			losses.append([loss_c1, loss_c2, loss_c3, loss_dr1, loss_dr2, loss_dr3, loss_dv, loss_ir1, loss_ir2, loss_ir3, loss_iv])

		loss_c1 = sum([l[0] for l in losses])
		loss_c2 = sum([l[1] for l in losses])
		loss_c3 = sum([l[2] for l in losses])
		loss_dr1 = sum([l[3] for l in losses])
		loss_dr2 = sum([l[4] for l in losses])
		loss_dr3 = sum([l[5] for l in losses])
		loss_dv = sum([l[6] for l in losses])
		loss_ir1 = sum([l[7] for l in losses])
		loss_ir2 = sum([l[8] for l in losses])
		loss_ir3 = sum([l[9] for l in losses])
		loss_iv = sum([l[10] for l in losses])
		
		loss = w_c1*loss_c1 + w_c2*loss_c2 + w_c3*loss_c3 + w_dr1*loss_dr1 + w_dr2*loss_dr2 + w_dr3*loss_dr3 + \
				w_dv*loss_dv + w_ir1*loss_ir1 + w_ir2*loss_ir2 + w_ir3*loss_ir3 + w_iv*loss_iv
		return loss, loss_c1, loss_c2, loss_c3, loss_dr1, loss_dr2, loss_dr3, loss_dv, loss_ir1, loss_ir2, loss_ir3, loss_iv
	
def LBFGS_batch_optimize(data_sources, batch_proportion = 0.01, save_path = "", model_names = None, loss_names = None, print_every = 1, **kwargs):
	shuffled = lambda dataset: [tf.data.Dataset.from_tensor_slices(tuple(d)).shuffle(buffer_size = max(d[0].shape[0], 1), \
                                                        reshuffle_each_iteration = True).batch(max(int(batch_proportion*d[0].shape[0]), 1)) \
                                        for d in dataset]
	data_s = shuffled([d for d_s in data_sources for d in d_s])
	for d in zip(*data_s): # d: [d_1_c, d_1_i, d_1_d, d_2_c, ...]
		data_sources = [list(d[i*3])+list(d[i*3+1])+list(d[i*3+2]) for i in range(n_sources)]
		LBFGS_optimizer = LBFGS([direct_model, inverse_model])
		LBFGS_optimizer.set_data(data_sources)
		LBFGS_optimizer.optimize(save_path = "models/{}/{}/".format(NAME, save_path), model_names = model_names, loss_names = loss_names, print_loss = True, **kwargs)
	del LBFGS_optimizer

In [8]:
from scipy.io import loadmat

x_s = np.linspace(25.0, 225.0, 9) / x0
z_s = 5.0 / z0
t_s = 0.05 / t0
n_sources = len(x_s)

domain = [0, 250]
T_max = 0.25
dx = 5
dt = 0.0025

x_d = np.linspace(5.0, 250.0, 50) / x0
z_d = np.array([5.0]) / z0
t_d = np.linspace(0.0, 0.2475, 100) / t0

n_c = 1000
x_c = transform(np.random.random((n_c, 1)), *domain) / x0
z_c = transform(np.random.random((n_c, 1)), *domain) / z0
t_c = transform(np.random.random((n_c, 1)), 0, T_max) / t0

n_s = 100
x_c_source = np.vstack([transform(np.random.random((n_s, 1)), xi-5, xi+5) for xi in x_s]) / x0
z_c_source = np.vstack([transform(np.random.random((n_s, 1)), z_s-5, z_s+5) for i in x_s]) / z0
t_c_source = np.vstack([transform(np.random.random((n_s, 1)), t_s-0.05, t_s+0.05) for i in x_s]) / t0

x_c = np.vstack([x_c, x_c_source]) / x0
z_c = np.vstack([z_c, z_c_source]) / z0
t_c = np.vstack([t_c, t_c_source]) / t0

n_i = 2000
x_i = transform(np.random.random((n_i, 1)), *domain) / x0
z_i = transform(np.random.random((n_i, 1)), *domain) / z0
t_i = np.zeros_like(x_i) / t0
p_i = np.zeros_like(x_i) / p0
u_i = np.zeros_like(x_i) / u0
v_i = np.zeros_like(x_i) / v0

# txz_c = tensor_grid([t_c, x_c, z_c])
txz_d = tensor_grid([t_d, x_d, z_d])

map_to_tf_float32 = lambda x: list(map(lambda y: tf.constant(y, dtype = tf.float32), x))

data_sources = []
for x in x_s:
	p = loadmat("data/sr_p_{}.mat".format(int(x*x0/5)))["sr"].reshape((-1, 1)) / p0 * 1000
	v = loadmat("data/sr_vz_{}.mat".format(int(x*x0/5)))["sr"].reshape((-1, 1)) / v0
	d_i = map_to_tf_float32([x_i, z_i, t_i, p_i, u_i, v_i, x*np.ones_like(x_i), z_s*np.ones_like(z_i), t_s*np.ones_like(t_i)])
	d_c = map_to_tf_float32([x_c, z_c, t_c, x*np.ones_like(x_c), z_s*np.ones_like(z_c), t_s*np.ones_like(t_c)])
	d_d = map_to_tf_float32([txz_d[:, 1:2], txz_d[:, 2:3], txz_d[:, 0:1], p, v, x*np.ones_like(txz_d[:, 1:2]), z_s*np.ones_like(txz_d[:, 2:3]), t_s*np.ones_like(txz_d[:, 0:1])])
	data_sources.append([d_c, d_i, d_d])

In [9]:
i = 1
optimizer = keras.optimizers.Adam(lr = 1e-3)

w_c1, w_c2, w_c3 = [0]*3
w_dr1, w_dr2, w_dr3 = [0]*3
w_ir1, w_ir2, w_ir3 = [0]*3
w_dv = 1e2
w_iv = 0

w_zero = 0.01
threshold = 1e-4

train(data_sources, 2000, batch_proportion = 1.0, print_every = 10, save_every = 100, save_path = "{}_adam".format(i))

2020/06/29, 22:42:38, Epoch: 10, Loss: 2.2023e+02, c1: 1.4765e+07, c2: 1.0056e+04, c3: 7.6340e+03, dr1: 8.6383e+09, dr2: 1.0115e+04, dr3: 7.6159e+03, 					  dv: 2.2023e+00, ir1: 1.3611e+10, ir2: 1.0019e+04, ir3: 7.6026e+03, iv: 9.1025e+00
2020/06/29, 22:43:07, Epoch: 20, Loss: 1.3031e+02, c1: 1.4762e+07, c2: 1.0182e+04, c3: 7.6168e+03, dr1: 8.6531e+09, dr2: 1.0220e+04, dr3: 7.5839e+03, 					  dv: 1.3031e+00, ir1: 1.3636e+10, ir2: 1.0103e+04, ir3: 7.5781e+03, iv: 8.1644e+00
2020/06/29, 22:43:36, Epoch: 30, Loss: 9.1853e+01, c1: 1.4621e+07, c2: 1.0223e+04, c3: 7.5461e+03, dr1: 8.5921e+09, dr2: 1.0250e+04, dr3: 7.5099e+03, 					  dv: 9.1853e-01, ir1: 1.3530e+10, ir2: 1.0125e+04, ir3: 7.5082e+03, iv: 7.7143e+00
2020/06/29, 22:43:59, Epoch: 40, Loss: 6.9833e+01, c1: 1.4429e+07, c2: 1.0196e+04, c3: 7.4526e+03, dr1: 8.4903e+09, dr2: 1.0219e+04, dr3: 7.4205e+03, 					  dv: 6.9833e-01, ir1: 1.3373e+10, ir2: 1.0096e+04, ir3: 7.4193e+03, iv: 7.4111e+00
2020/06/29, 22:44:17, Epoch: 50, Loss: 5.564

INFO:tensorflow:Assets written to: models/0628_1_direct_model_with_source_input_siren_scale_equation_weighted_loss/1_adam/direct_model/assets
INFO:tensorflow:Assets written to: models/0628_1_direct_model_with_source_input_siren_scale_equation_weighted_loss/1_adam/inverse_model/assets
2020/06/29, 22:49:25, Epoch: 210, Loss: 1.0548e+01, c1: 1.1994e+07, c2: 9.0752e+03, c3: 5.8137e+03, dr1: 7.0944e+09, dr2: 9.1602e+03, dr3: 5.7934e+03, 					  dv: 1.0548e-01, ir1: 1.1245e+10, ir2: 9.0292e+03, ir3: 5.7854e+03, iv: 5.1777e+00
2020/06/29, 22:49:55, Epoch: 220, Loss: 1.0042e+01, c1: 1.1901e+07, c2: 9.0170e+03, c3: 5.7559e+03, dr1: 7.0399e+09, dr2: 9.1031e+03, dr3: 5.7364e+03, 					  dv: 1.0042e-01, ir1: 1.1160e+10, ir2: 8.9734e+03, ir3: 5.7287e+03, iv: 5.0968e+00
2020/06/29, 22:50:25, Epoch: 230, Loss: 9.5926e+00, c1: 1.1812e+07, c2: 8.9597e+03, c3: 5.7009e+03, dr1: 6.9877e+09, dr2: 9.0470e+03, dr3: 5.6825e+03, 					  dv: 9.5926e-02, ir1: 1.1078e+10, ir2: 8.9184e+03, ir3: 5.6750e+03, iv: 5.0188

INFO:tensorflow:Assets written to: models/0628_1_direct_model_with_source_input_siren_scale_equation_weighted_loss/1_adam/inverse_model/assets
2020/06/29, 23:00:54, Epoch: 510, Loss: 5.4053e+00, c1: 1.0236e+07, c2: 7.7686e+03, c3: 4.8280e+03, dr1: 6.0457e+09, dr2: 7.8538e+03, dr3: 4.8297e+03, 					  dv: 5.4053e-02, ir1: 9.5906e+09, ir2: 7.7475e+03, ir3: 4.8148e+03, iv: 3.6215e+00
2020/06/29, 23:01:13, Epoch: 520, Loss: 5.4064e+00, c1: 1.0200e+07, c2: 7.7376e+03, c3: 4.8103e+03, dr1: 6.0239e+09, dr2: 7.8220e+03, dr3: 4.8117e+03, 					  dv: 5.4064e-02, ir1: 9.5561e+09, ir2: 7.7169e+03, ir3: 4.7970e+03, iv: 3.5900e+00
2020/06/29, 23:01:33, Epoch: 530, Loss: 5.3759e+00, c1: 1.0164e+07, c2: 7.7073e+03, c3: 4.7933e+03, dr1: 6.0027e+09, dr2: 7.7910e+03, dr3: 4.7943e+03, 					  dv: 5.3759e-02, ir1: 9.5227e+09, ir2: 7.6870e+03, ir3: 4.7798e+03, iv: 3.5594e+00
2020/06/29, 23:01:52, Epoch: 540, Loss: 5.3300e+00, c1: 1.0128e+07, c2: 7.6776e+03, c3: 4.7765e+03, dr1: 5.9820e+09, dr2: 7.7607e+03, dr3

2020/06/29, 23:10:28, Epoch: 810, Loss: 4.8040e+00, c1: 9.3782e+06, c2: 7.0390e+03, c3: 4.4476e+03, dr1: 5.5583e+09, dr2: 7.1031e+03, dr3: 4.4349e+03, 					  dv: 4.8040e-02, ir1: 8.8159e+09, ir2: 7.0248e+03, ir3: 4.4326e+03, iv: 2.9444e+00
2020/06/29, 23:10:47, Epoch: 820, Loss: 4.7850e+00, c1: 9.3563e+06, c2: 7.0204e+03, c3: 4.4387e+03, dr1: 5.5465e+09, dr2: 7.0837e+03, dr3: 4.4255e+03, 					  dv: 4.7850e-02, ir1: 8.7965e+09, ir2: 7.0062e+03, ir3: 4.4235e+03, iv: 2.9287e+00
2020/06/29, 23:11:06, Epoch: 830, Loss: 4.7660e+00, c1: 9.3345e+06, c2: 7.0021e+03, c3: 4.4299e+03, dr1: 5.5348e+09, dr2: 7.0648e+03, dr3: 4.4162e+03, 					  dv: 4.7660e-02, ir1: 8.7774e+09, ir2: 6.9880e+03, ir3: 4.4144e+03, iv: 2.9134e+00
2020/06/29, 23:11:26, Epoch: 840, Loss: 4.7714e+00, c1: 9.3132e+06, c2: 6.9842e+03, c3: 4.4213e+03, dr1: 5.5233e+09, dr2: 7.0461e+03, dr3: 4.4072e+03, 					  dv: 4.7714e-02, ir1: 8.7585e+09, ir2: 6.9700e+03, ir3: 4.4056e+03, iv: 2.8984e+00
2020/06/29, 23:11:45, Epoch: 850, Loss: 

2020/06/29, 23:20:23, Epoch: 1120, Loss: 4.5387e+00, c1: 8.8027e+06, c2: 6.5688e+03, c3: 4.2315e+03, dr1: 5.2541e+09, dr2: 6.6141e+03, dr3: 4.2001e+03, 					  dv: 4.5387e-02, ir1: 8.3214e+09, ir2: 6.5526e+03, ir3: 4.2048e+03, iv: 2.5723e+00
2020/06/29, 23:20:42, Epoch: 1130, Loss: 4.5816e+00, c1: 8.7873e+06, c2: 6.5567e+03, c3: 4.2262e+03, dr1: 5.2462e+09, dr2: 6.6014e+03, dr3: 4.1943e+03, 					  dv: 4.5816e-02, ir1: 8.3085e+09, ir2: 6.5403e+03, ir3: 4.1991e+03, iv: 2.5634e+00
2020/06/29, 23:21:01, Epoch: 1140, Loss: 4.5217e+00, c1: 8.7721e+06, c2: 6.5447e+03, c3: 4.2210e+03, dr1: 5.2382e+09, dr2: 6.5889e+03, dr3: 4.1884e+03, 					  dv: 4.5217e-02, ir1: 8.2958e+09, ir2: 6.5281e+03, ir3: 4.1934e+03, iv: 2.5546e+00
2020/06/29, 23:21:20, Epoch: 1150, Loss: 4.5662e+00, c1: 8.7569e+06, c2: 6.5327e+03, c3: 4.2157e+03, dr1: 5.2303e+09, dr2: 6.5765e+03, dr3: 4.1826e+03, 					  dv: 4.5662e-02, ir1: 8.2832e+09, ir2: 6.5159e+03, ir3: 4.1877e+03, iv: 2.5460e+00
2020/06/29, 23:21:39, Epoch: 1160, L

2020/06/29, 23:32:10, Epoch: 1430, Loss: 4.4801e+00, c1: 8.3958e+06, c2: 6.2575e+03, c3: 4.0984e+03, dr1: 5.0426e+09, dr2: 6.2919e+03, dr3: 4.0489e+03, 					  dv: 4.4801e-02, ir1: 7.9883e+09, ir2: 6.2383e+03, ir3: 4.0608e+03, iv: 2.3583e+00
2020/06/29, 23:32:40, Epoch: 1440, Loss: 4.4626e+00, c1: 8.3846e+06, c2: 6.2493e+03, c3: 4.0952e+03, dr1: 5.0369e+09, dr2: 6.2834e+03, dr3: 4.0451e+03, 					  dv: 4.4626e-02, ir1: 7.9793e+09, ir2: 6.2300e+03, ir3: 4.0573e+03, iv: 2.3532e+00
2020/06/29, 23:33:10, Epoch: 1450, Loss: 4.4835e+00, c1: 8.3736e+06, c2: 6.2413e+03, c3: 4.0922e+03, dr1: 5.0314e+09, dr2: 6.2752e+03, dr3: 4.0414e+03, 					  dv: 4.4835e-02, ir1: 7.9706e+09, ir2: 6.2218e+03, ir3: 4.0539e+03, iv: 2.3482e+00
2020/06/29, 23:33:40, Epoch: 1460, Loss: 4.4803e+00, c1: 8.3627e+06, c2: 6.2334e+03, c3: 4.0892e+03, dr1: 5.0259e+09, dr2: 6.2670e+03, dr3: 4.0377e+03, 					  dv: 4.4803e-02, ir1: 7.9619e+09, ir2: 6.2138e+03, ir3: 4.0506e+03, iv: 2.3433e+00
2020/06/29, 23:34:11, Epoch: 1470, L

2020/06/29, 23:46:28, Epoch: 1740, Loss: 4.4136e+00, c1: 8.0954e+06, c2: 6.0539e+03, c3: 4.0250e+03, dr1: 4.8927e+09, dr2: 6.0802e+03, dr3: 3.9557e+03, 					  dv: 4.4136e-02, ir1: 7.7542e+09, ir2: 6.0307e+03, ir3: 3.9771e+03, iv: 2.2386e+00
2020/06/29, 23:46:50, Epoch: 1750, Loss: 4.3852e+00, c1: 8.0875e+06, c2: 6.0488e+03, c3: 4.0233e+03, dr1: 4.8886e+09, dr2: 6.0749e+03, dr3: 3.9534e+03, 					  dv: 4.3852e-02, ir1: 7.7479e+09, ir2: 6.0255e+03, ir3: 3.9751e+03, iv: 2.2359e+00
2020/06/29, 23:47:09, Epoch: 1760, Loss: 4.4230e+00, c1: 8.0798e+06, c2: 6.0436e+03, c3: 4.0218e+03, dr1: 4.8846e+09, dr2: 6.0695e+03, dr3: 3.9512e+03, 					  dv: 4.4230e-02, ir1: 7.7417e+09, ir2: 6.0203e+03, ir3: 3.9731e+03, iv: 2.2333e+00
2020/06/29, 23:47:35, Epoch: 1770, Loss: 4.4236e+00, c1: 8.0722e+06, c2: 6.0386e+03, c3: 4.0203e+03, dr1: 4.8807e+09, dr2: 6.0644e+03, dr3: 3.9491e+03, 					  dv: 4.4236e-02, ir1: 7.7357e+09, ir2: 6.0152e+03, ir3: 3.9713e+03, iv: 2.2308e+00
2020/06/29, 23:48:05, Epoch: 1780, L

In [10]:
i = 1
optimizer = keras.optimizers.Adam(lr = 2e-3)

w_c1, w_c2, w_c3 = [0]*3
w_dr1, w_dr2, w_dr3 = [0]*3
w_ir1, w_ir2, w_ir3 = [0]*3
w_dv = 1e2
w_iv = 1e-2

w_zero = 0.01
threshold = 1e-4

train(data_sources, 2000, batch_proportion = 1.0, print_every = 10, save_every = 100, save_path = "{}_adam".format(i))

2020/06/29, 23:59:40, Epoch: 10, Loss: 5.1247e+00, c1: 6.8021e+06, c2: 5.1053e+03, c3: 3.4636e+03, dr1: 4.1130e+09, dr2: 5.1304e+03, dr3: 3.4238e+03, 					  dv: 5.1091e-02, ir1: 6.5085e+09, ir2: 5.1012e+03, ir3: 3.4478e+03, iv: 1.5563e+00
2020/06/30, 00:00:10, Epoch: 20, Loss: 4.6093e+00, c1: 5.9415e+06, c2: 4.4383e+03, c3: 3.0853e+03, dr1: 3.5825e+09, dr2: 4.4689e+03, dr3: 3.0582e+03, 					  dv: 4.5976e-02, ir1: 5.6817e+09, ir2: 4.4429e+03, ir3: 3.0815e+03, iv: 1.1726e+00
2020/06/30, 00:00:41, Epoch: 30, Loss: 4.3824e+00, c1: 5.1771e+06, c2: 3.8371e+03, c3: 2.7757e+03, dr1: 3.1282e+09, dr2: 3.8736e+03, dr3: 2.7433e+03, 					  dv: 4.3738e-02, ir1: 4.9837e+09, ir2: 3.8530e+03, ir3: 2.7582e+03, iv: 8.6698e-01
2020/06/30, 00:01:11, Epoch: 40, Loss: 4.3664e+00, c1: 4.5843e+06, c2: 3.3348e+03, c3: 2.4908e+03, dr1: 2.7733e+09, dr2: 3.3855e+03, dr3: 2.4577e+03, 					  dv: 4.3598e-02, ir1: 4.4279e+09, ir2: 3.3627e+03, ir3: 2.4675e+03, iv: 6.6627e-01
2020/06/30, 00:01:42, Epoch: 50, Loss: 4.347

2020/06/30, 00:15:17, Epoch: 320, Loss: 3.8167e+00, c1: 9.1112e+05, c2: 6.2265e+02, c3: 4.8938e+02, dr1: 5.2273e+08, dr2: 6.2483e+02, dr3: 4.8830e+02, 					  dv: 3.8164e-02, ir1: 8.3340e+08, ir2: 6.2695e+02, ir3: 4.8696e+02, iv: 2.6127e-02
2020/06/30, 00:15:46, Epoch: 330, Loss: 3.7972e+00, c1: 8.8662e+05, c2: 6.1468e+02, c3: 4.6896e+02, dr1: 5.0807e+08, dr2: 6.1568e+02, dr3: 4.6676e+02, 					  dv: 3.7970e-02, ir1: 8.1067e+08, ir2: 6.1776e+02, ir3: 4.6624e+02, iv: 2.4904e-02
2020/06/30, 00:16:05, Epoch: 340, Loss: 3.7746e+00, c1: 8.6320e+05, c2: 6.0569e+02, c3: 4.5025e+02, dr1: 4.9443e+08, dr2: 6.0545e+02, dr3: 4.4624e+02, 					  dv: 3.7743e-02, ir1: 7.8941e+08, ir2: 6.0772e+02, ir3: 4.4647e+02, iv: 2.3735e-02
2020/06/30, 00:16:25, Epoch: 350, Loss: 3.7513e+00, c1: 8.3730e+05, c2: 5.9547e+02, c3: 4.3230e+02, dr1: 4.8006e+08, dr2: 5.9407e+02, dr3: 4.2681e+02, 					  dv: 3.7511e-02, ir1: 7.6719e+08, ir2: 5.9649e+02, ir3: 4.2779e+02, iv: 2.2593e-02
2020/06/30, 00:16:44, Epoch: 360, Loss: 

2020/06/30, 00:25:24, Epoch: 630, Loss: 3.3046e+00, c1: 4.1026e+05, c2: 3.1010e+02, c3: 1.9823e+02, dr1: 2.4258e+08, dr2: 3.1742e+02, dr3: 1.9579e+02, 					  dv: 3.3045e-02, ir1: 3.7930e+08, ir2: 3.1352e+02, ir3: 1.9498e+02, iv: 1.0656e-02
2020/06/30, 00:25:43, Epoch: 640, Loss: 3.2800e+00, c1: 4.0486e+05, c2: 3.0388e+02, c3: 1.9565e+02, dr1: 2.3904e+08, dr2: 3.1211e+02, dr3: 1.9338e+02, 					  dv: 3.2799e-02, ir1: 3.7356e+08, ir2: 3.0863e+02, ir3: 1.9254e+02, iv: 1.0488e-02
2020/06/30, 00:26:02, Epoch: 650, Loss: 3.2801e+00, c1: 3.9986e+05, c2: 2.9806e+02, c3: 1.9290e+02, dr1: 2.3572e+08, dr2: 3.0714e+02, dr3: 1.9086e+02, 					  dv: 3.2800e-02, ir1: 3.6814e+08, ir2: 3.0418e+02, ir3: 1.9008e+02, iv: 1.0629e-02
2020/06/30, 00:26:21, Epoch: 660, Loss: 3.2639e+00, c1: 3.9564e+05, c2: 2.9339e+02, c3: 1.9098e+02, dr1: 2.3281e+08, dr2: 3.0327e+02, dr3: 1.8900e+02, 					  dv: 3.2638e-02, ir1: 3.6370e+08, ir2: 3.0050e+02, ir3: 1.8823e+02, iv: 1.0546e-02
2020/06/30, 00:26:40, Epoch: 670, Loss: 

2020/06/30, 00:37:33, Epoch: 940, Loss: 3.0793e+00, c1: 3.0898e+05, c2: 2.2823e+02, c3: 1.4392e+02, dr1: 1.8157e+08, dr2: 2.3476e+02, dr3: 1.4417e+02, 					  dv: 3.0792e-02, ir1: 2.8898e+08, ir2: 2.3524e+02, ir3: 1.4446e+02, iv: 1.2201e-02
2020/06/30, 00:37:52, Epoch: 950, Loss: 2.9969e+00, c1: 3.0507e+05, c2: 2.2552e+02, c3: 1.4213e+02, dr1: 1.7957e+08, dr2: 2.3205e+02, dr3: 1.4257e+02, 					  dv: 2.9968e-02, ir1: 2.8573e+08, ir2: 2.3229e+02, ir3: 1.4297e+02, iv: 1.1405e-02
2020/06/30, 00:38:11, Epoch: 960, Loss: 2.9929e+00, c1: 3.0106e+05, c2: 2.2224e+02, c3: 1.4008e+02, dr1: 1.7759e+08, dr2: 2.2870e+02, dr3: 1.4059e+02, 					  dv: 2.9927e-02, ir1: 2.8245e+08, ir2: 2.2911e+02, ir3: 1.4106e+02, iv: 1.1380e-02
2020/06/30, 00:38:31, Epoch: 970, Loss: 2.9857e+00, c1: 2.9681e+05, c2: 2.1920e+02, c3: 1.3813e+02, dr1: 1.7531e+08, dr2: 2.2534e+02, dr3: 1.3867e+02, 					  dv: 2.9856e-02, ir1: 2.7867e+08, ir2: 2.2599e+02, ir3: 1.3923e+02, iv: 1.1414e-02
2020/06/30, 00:38:50, Epoch: 980, Loss: 

2020/06/30, 00:49:42, Epoch: 1250, Loss: 2.7857e+00, c1: 1.9423e+05, c2: 1.5416e+02, c3: 7.0831e+01, dr1: 1.1387e+08, dr2: 1.5309e+02, dr3: 7.1230e+01, 					  dv: 2.7855e-02, ir1: 1.8268e+08, ir2: 1.5388e+02, ir3: 7.2325e+01, iv: 1.1546e-02
2020/06/30, 00:50:01, Epoch: 1260, Loss: 2.8094e+00, c1: 1.9080e+05, c2: 1.5180e+02, c3: 6.8806e+01, dr1: 1.1187e+08, dr2: 1.5065e+02, dr3: 6.9177e+01, 					  dv: 2.8093e-02, ir1: 1.7958e+08, ir2: 1.5144e+02, ir3: 7.0275e+01, iv: 1.1856e-02
2020/06/30, 00:50:20, Epoch: 1270, Loss: 2.7851e+00, c1: 1.8683e+05, c2: 1.4933e+02, c3: 6.6277e+01, dr1: 1.0970e+08, dr2: 1.4818e+02, dr3: 6.6670e+01, 					  dv: 2.7850e-02, ir1: 1.7616e+08, ir2: 1.4896e+02, ir3: 6.7786e+01, iv: 1.1614e-02
2020/06/30, 00:50:40, Epoch: 1280, Loss: 2.7768e+00, c1: 1.8359e+05, c2: 1.4680e+02, c3: 6.4292e+01, dr1: 1.0779e+08, dr2: 1.4561e+02, dr3: 6.4675e+01, 					  dv: 2.7767e-02, ir1: 1.7312e+08, ir2: 1.4649e+02, ir3: 6.5774e+01, iv: 1.1638e-02
2020/06/30, 00:50:59, Epoch: 1290, L

2020/06/30, 01:02:23, Epoch: 1560, Loss: 2.6603e+00, c1: 1.0927e+05, c2: 9.1810e+01, c3: 3.8457e+01, dr1: 6.5742e+07, dr2: 8.9244e+01, dr3: 3.8216e+01, 					  dv: 2.6602e-02, ir1: 1.0493e+08, ir2: 8.9646e+01, ir3: 3.8939e+01, iv: 1.3200e-02
2020/06/30, 01:02:52, Epoch: 1570, Loss: 2.6726e+00, c1: 1.0729e+05, c2: 9.0114e+01, c3: 3.8115e+01, dr1: 6.4588e+07, dr2: 8.7606e+01, dr3: 3.7837e+01, 					  dv: 2.6725e-02, ir1: 1.0315e+08, ir2: 8.7943e+01, ir3: 3.8546e+01, iv: 1.3304e-02
2020/06/30, 01:03:22, Epoch: 1580, Loss: 2.6566e+00, c1: 1.0531e+05, c2: 8.8436e+01, c3: 3.7723e+01, dr1: 6.3450e+07, dr2: 8.5909e+01, dr3: 3.7453e+01, 					  dv: 2.6564e-02, ir1: 1.0133e+08, ir2: 8.6250e+01, ir3: 3.8149e+01, iv: 1.3267e-02
2020/06/30, 01:03:51, Epoch: 1590, Loss: 2.6553e+00, c1: 1.0337e+05, c2: 8.6734e+01, c3: 3.7373e+01, dr1: 6.2310e+07, dr2: 8.4273e+01, dr3: 3.7080e+01, 					  dv: 2.6551e-02, ir1: 9.9579e+07, ir2: 8.4594e+01, ir3: 3.7769e+01, iv: 1.3334e-02
2020/06/30, 01:04:20, Epoch: 1600, L

2020/06/30, 01:13:57, Epoch: 1870, Loss: 2.5769e+00, c1: 6.4317e+04, c2: 4.9753e+01, c3: 2.7870e+01, dr1: 3.8439e+07, dr2: 4.9212e+01, dr3: 2.7399e+01, 					  dv: 2.5768e-02, ir1: 6.1364e+07, ir2: 4.9490e+01, ir3: 2.7802e+01, iv: 1.3923e-02
2020/06/30, 01:14:16, Epoch: 1880, Loss: 2.5804e+00, c1: 6.3290e+04, c2: 4.8864e+01, c3: 2.7506e+01, dr1: 3.7784e+07, dr2: 4.8347e+01, dr3: 2.7032e+01, 					  dv: 2.5802e-02, ir1: 6.0336e+07, ir2: 4.8638e+01, ir3: 2.7453e+01, iv: 1.3990e-02
2020/06/30, 01:14:35, Epoch: 1890, Loss: 2.5702e+00, c1: 6.2187e+04, c2: 4.7910e+01, c3: 2.7135e+01, dr1: 3.7093e+07, dr2: 4.7408e+01, dr3: 2.6665e+01, 					  dv: 2.5700e-02, ir1: 5.9230e+07, ir2: 4.7698e+01, ir3: 2.7082e+01, iv: 1.3941e-02
2020/06/30, 01:14:54, Epoch: 1900, Loss: 2.5692e+00, c1: 6.1197e+04, c2: 4.7088e+01, c3: 2.6788e+01, dr1: 3.6491e+07, dr2: 4.6596e+01, dr3: 2.6324e+01, 					  dv: 2.5690e-02, ir1: 5.8275e+07, ir2: 4.6899e+01, ir3: 2.6748e+01, iv: 1.4052e-02
INFO:tensorflow:Assets written to: m

In [ ]:
i = 2

optimizer.learning_rate = 5e-3

w_c1, w_c2, w_c3 = [0]*3
w_dr1, w_dr2, w_dr3 = [0]*3
w_ir1, w_ir2, w_ir3 = [0]*3
w_dv = 1e2
w_iv = 1e-5

w_zero = 0.001
threshold = 1e-4

train(data_sources, 2000, batch_proportion = 1.0, print_every = 10, save_every = 100, save_path = "{}_adam".format(i))

In [ ]:
save_path = "2_adam"
direct_model = keras.models.load_model("models/{}/{}/direct_model".format(NAME, save_path))
inverse_model = keras.models.load_model("models/{}/{}/inverse_model".format(NAME, save_path))

In [ ]:
i = 3

optimizer = keras.optimizers.Adam(lr = 1e-3)


w_c1, w_c2, w_c3 = [0]*3
w_dr1, w_dr2, w_dr3 = [0]*3
w_ir1, w_ir2, w_ir3 = [0]*3
w_dv = 1e2
w_iv = 1e-1

w_zero = 0.001
threshold = 1e-4

train(data_sources, 2000, batch_proportion = 1.0, print_every = 10, save_every = 100, save_path = "{}_adam".format(i))

In [ ]:
# i = 1
# w_c1, w_c2, w_c3 = [0]*3
# w_dr1, w_dr2, w_dr3 = [0]*3
# w_ir1, w_ir2, w_ir3 = [0]*3
# w_dv = 1e3
# w_iv = 1e-5

# w_zero = 0.1
# threshold = 1e-4

# LBFGS_batch_optimize(data_sources, batch_proportion = 1.0, save_path = "{}_lbfgs".format(i), 
# 					model_names = ["direct_model", "inverse_model"],
# 					loss_names = ["c1", "c2", "c3", "dr1", "dr2", "dr3", "dv", "ir1", "ir2", "ir3", "iv"],
# 					 num_correction_pairs=10,
# 					 tolerance=1e-31, 
# 					 x_tolerance=1e-31, 
# 					 f_relative_tolerance=1e-31,
# 					 initial_inverse_hessian_estimate=None, 
# 					 max_iterations=1)

In [ ]:
x_test = np.linspace(1.0, 250.0, 250) / x0
z_test = np.linspace(1.0, 250.0, 499) / z0
xz_test = tensor_grid([z_test, x_test])
xz_test[:, [0, 1]] = xz_test[:, [1, 0]]

c_pred = inverse_model(tf.constant(xz_test, dtype = tf.float32), training = False)*c0
def c_func(x, z):
	return np.piecewise(z, [z > 150, (z >= 100) & (z <= 150), z < 100], [1500, 3500, 1500])
c_true = c_func(xz_test[:, 0:1]*x0, xz_test[:, 1:2]*z0)

import matplotlib as mpl
from matplotlib.cm import cool
from matplotlib.colors import Normalize

from mpl_toolkits.axes_grid1 import make_axes_locatable

cmap = cool
norm = Normalize(vmin=1000, vmax=5000)

X, Z = np.meshgrid(x_test, z_test)
fig, ax = plt.subplots(1, 3, figsize = (15, 5))
fig.subplots_adjust(right = 1.0)

im0 = ax[0].contourf(X, Z, c_true.reshape((len(z_test), len(x_test))), cmap = cmap, norm = norm)
ax[0].set_title("true")
divider = make_axes_locatable(ax[0])
cax = divider.append_axes('right', size='5%', pad=0.05)
mpl.colorbar.ColorbarBase(cax, cmap = cmap, norm = norm, orientation='vertical')

norm = mpl.colors.Normalize(vmin=0, vmax=5000)
im1 = ax[1].contourf(X, Z, c_pred.numpy().reshape((len(z_test), len(x_test))), cmap = cmap, norm = norm)
ax[1].set_title("pred")
ax[1].set_xlabel("")
divider = make_axes_locatable(ax[1])
cax = divider.append_axes('right', size='5%', pad=0.05)
mpl.colorbar.ColorbarBase(cax, cmap = cmap, norm = norm, orientation='vertical')

norm = mpl.colors.Normalize(vmin=-1e-2, vmax=1e-2)
im2 = ax[2].contourf(X, Z, c_true.reshape((len(z_test), len(x_test)))-c_pred.numpy().reshape((len(z_test), len(x_test))), cmap = cmap, norm = norm)
ax[2].set_title("error")
divider = make_axes_locatable(ax[2])
cax = divider.append_axes('right', size='5%', pad=0.05)
mpl.colorbar.ColorbarBase(cax, cmap = cmap, norm = norm, orientation='vertical')

plt.show()

In [ ]:
i = 1
p = loadmat("data/sr_p_{}.mat".format(int(x_s[i]*x0/5)))["sr"]*1000
v = loadmat("data/sr_vz_{}.mat".format(int(x_s[i]*x0/5)))["sr"]

x_test = np.linspace(5.0, 250.0, 50) / x0
# z_test = np.linspace(1.0, 250.0, 250)
z_test = np.array([5]) / z0
t_test = np.linspace(0.0, 0.2475, 100) / t0
xzt_test = tensor_grid([t_test, x_test, z_test])
xzt_test[:, [0, 1, 2]] = xzt_test[:, [1, 2, 0]]

xzts = np.hstack([xzt_test, x_s[i]*np.ones_like(xzt_test[:, 0:1])])
out = direct_model(tf.constant(xzts, dtype = tf.float32))

cmap = cool
norm = Normalize(vmin=-1, vmax=1)

X, T = np.meshgrid(x_test, t_test)
fig, ax = plt.subplots(1, 3, figsize = (15, 5))
fig.subplots_adjust(right = 1.0)

im0 = ax[0].contourf(X, T, p, cmap = cmap, norm = norm, levels = 1000)
ax[0].set_title("p, true")
divider = make_axes_locatable(ax[0])
cax = divider.append_axes('right', size='5%', pad=0.05)
mpl.colorbar.ColorbarBase(cax, cmap = cmap, norm = norm, orientation='vertical')

norm = Normalize(vmin=-1.0, vmax=1.0)
im1 = ax[1].contourf(X, T, out[:, 0:1].numpy().reshape((len(t_test), len(x_test))), cmap = cmap, norm = norm, levels = 1000)
ax[1].set_title("p, pred")
ax[1].set_xlabel("")
divider = make_axes_locatable(ax[1])
cax = divider.append_axes('right', size='5%', pad=0.05)
mpl.colorbar.ColorbarBase(cax, cmap = cmap, norm = norm, orientation='vertical')

norm = mpl.colors.Normalize(vmin=-2e-1, vmax=2e-1)
im2 = ax[2].contourf(X, T, p-out[:, 0:1].numpy().reshape((len(t_test), len(x_test))), cmap = cmap, norm = norm, levels = 1000)
ax[2].set_title("p, error")
divider = make_axes_locatable(ax[2])
cax = divider.append_axes('right', size='5%', pad=0.05)
mpl.colorbar.ColorbarBase(cax, cmap = cmap, norm = norm, orientation='vertical')

plt.show()

In [ ]:
out